# Landsat Collections

In [1]:
import ee
ee.Initialize()

In [2]:
from geetools import collection, tools

In [3]:
import ipygee as ui

## Default values

In [4]:
collection.landsat.DEFAULTS

{1: {'process': 'RAW', 'sensor': 'MSS'},
 2: {'process': 'RAW', 'sensor': 'MSS'},
 3: {'process': 'RAW', 'sensor': 'MSS'},
 4: {'process': 'SR', 'sensor': 'TM'},
 5: {'process': 'SR', 'sensor': 'TM'},
 7: {'process': 'SR', 'sensor': 'ETM'},
 8: {'process': 'SR', 'sensor': 'OLI'}}

This means that the default collection for Landsat 1 is RAW and MSS, for Landsat 5 is SR and TM

# HELPERS

In [5]:
def print_values(image):
    print(tools.image.getValue(image, image.geometry().centroid(1), 30, side='client'))

## Landsat 8 SR

In [6]:
l8sr = collection.Landsat(8, 'SR')

## Information about the collection

In [7]:
collection.info(l8sr)

{'spacecraft': 'LANDSAT',
 'id': 'LANDSAT/LC08/C01/T1_SR',
 'bands': ['B1',
  'B2',
  'B3',
  'B4',
  'B5',
  'B6',
  'B7',
  'B10',
  'B11',
  'sr_aerosol',
  'pixel_qa',
  'radsat_qa'],
 'band_names': ['aerosol',
  'blue',
  'green',
  'red',
  'nir',
  'swir',
  'swir2',
  'thermal',
  'thermal2',
  'sr_aerosol',
  'pixel_qa',
  'radsat_qa'],
 'ranges': {'B1': {'min': 0, 'max': 10000},
  'B2': {'min': 0, 'max': 10000},
  'B3': {'min': 0, 'max': 10000},
  'B4': {'min': 0, 'max': 10000},
  'B5': {'min': 0, 'max': 10000},
  'B6': {'min': 0, 'max': 10000},
  'B7': {'min': 0, 'max': 10000},
  'B10': {'min': 0, 'max': 10000},
  'B11': {'min': 0, 'max': 10000},
  'sr_aerosol': {'min': None, 'max': None},
  'pixel_qa': {'min': None, 'max': None},
  'radsat_qa': {'min': None, 'max': None}},
 'scales': {'B1': 30,
  'B2': 30,
  'B3': 30,
  'B4': 30,
  'B5': 30,
  'B6': 30,
  'B7': 30,
  'B10': 30,
  'B11': 30,
  'sr_aerosol': 30,
  'pixel_qa': 30,
  'radsat_qa': 30},
 'start_date': '2013-04-11

## get a band by id

In [8]:
l8sr.getBand('atmos_opacity')

In [9]:
l8sr.getBand('B1').name

'aerosol'

In [10]:
l8sr.getBand('red', 'name').id

'B4'

## Get one image out of the collection

In [11]:
l8sr_i = ee.Image(l8sr.collection.first())

In [12]:
print_values(l8sr_i)

{'B1': 8394, 'B10': 2639, 'B11': 2642, 'B2': 8329, 'B3': 8054, 'B4': 8031, 'B5': 7478, 'B6': 3754, 'B7': 3120, 'pixel_qa': 480, 'radsat_qa': 0, 'sr_aerosol': 8}


# Rename bands

In [13]:
l8sr_i.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B10',
 'B11',
 'sr_aerosol',
 'pixel_qa',
 'radsat_qa']

In [14]:
l8sr_renamed = l8sr.rename(l8sr_i, 'all')

In [15]:
l8sr_renamed.bandNames().getInfo()

['aerosol',
 'blue',
 'green',
 'red',
 'nir',
 'swir',
 'swir2',
 'thermal',
 'thermal2',
 'sr_aerosol',
 'pixel_qa',
 'radsat_qa']

## Add NDVI

In [16]:
l8sr_i_ndvi = l8sr.ndvi(l8sr_i)

In [17]:
print_values(l8sr_i_ndvi)

{'ndvi': -0.03156969811083914}


## Get Common Bands

In [18]:
l8toa = collection.Landsat8TOA()

In [19]:
for band in l8toa.bands:
    print(band.id, band.name)

B1 aerosol
B2 blue
B3 green
B4 red
B5 nir
B6 swir
B7 swir2
B8 pan
B9 cirrus
B10 thermal
B11 thermal2
BQA bqa


In [20]:
collection.getCommonBands(l8sr, l8toa, reference='all', match='name')

['nir',
 'red',
 'swir',
 'blue',
 'swir2',
 'thermal',
 'aerosol',
 'green',
 'thermal2']

## Get an Image

In [21]:
l8toa_i = ee.Image(l8toa.collection.first())

## Re-scale to match Landsat 8 TOA collection

For rescaling uses `min` and `max` information for the band that match in each collection

In [22]:
# SR
l8sr.ranges('all', False)

{'B1': {'min': 0, 'max': 10000},
 'B2': {'min': 0, 'max': 10000},
 'B3': {'min': 0, 'max': 10000},
 'B4': {'min': 0, 'max': 10000},
 'B5': {'min': 0, 'max': 10000},
 'B6': {'min': 0, 'max': 10000},
 'B7': {'min': 0, 'max': 10000},
 'B10': {'min': 0, 'max': 10000},
 'B11': {'min': 0, 'max': 10000},
 'sr_aerosol': {'min': None, 'max': None},
 'pixel_qa': {'min': None, 'max': None},
 'radsat_qa': {'min': None, 'max': None}}

In [23]:
# TOA
l8toa.ranges()

{'B1': {'min': 0, 'max': 1},
 'B2': {'min': 0, 'max': 1},
 'B3': {'min': 0, 'max': 1},
 'B4': {'min': 0, 'max': 1},
 'B5': {'min': 0, 'max': 1},
 'B6': {'min': 0, 'max': 1},
 'B7': {'min': 0, 'max': 1},
 'B8': {'min': 0, 'max': 1},
 'B9': {'min': 0, 'max': 1},
 'B10': {'min': 0, 'max': 1000},
 'B11': {'min': 0, 'max': 1000},
 'BQA': {'min': None, 'max': None}}

In [24]:
for band in l8sr.bands:
    print(band.name, band.id, band.precision)

aerosol B1 int16
blue B2 int16
green B3 int16
red B4 int16
nir B5 int16
swir B6 int16
swir2 B7 int16
thermal B10 int16
thermal2 B11 int16
sr_aerosol sr_aerosol uint8
pixel_qa pixel_qa uint16
radsat_qa radsat_qa uint16


In [25]:
# l8_i_scaled_toa = l8sr.rescale_all(l8sr_i, 8, 'TOA', drop=True)
l8_toa_scaled_sr = collection.rescale(l8toa_i, l8toa, l8sr, drop=False)
print_values(l8_toa_scaled_sr)

{'B1': 8138, 'B10': 2638, 'B11': 2642, 'B2': 7998, 'B3': 7182, 'B4': 7489, 'B5': 7538, 'B6': 3911, 'B7': 3132, 'B8': 0.7274518013000488, 'B9': 0.003796438919380307, 'BQA': 2800}


In [26]:
l8_i_scaled_toa = collection.rescale(l8toa_i, l8toa, l8sr, drop=True)
print_values(l8_i_scaled_toa)

{'B1': 8138, 'B10': 2638, 'B11': 2642, 'B2': 7998, 'B3': 7182, 'B4': 7489, 'B5': 7538, 'B6': 3911, 'B7': 3132}


In [27]:
l8_renamed_scaled_toa = collection.rescale(l8sr_renamed, l8sr, l8toa, renamed=True, drop=True)
print_values(l8_renamed_scaled_toa)

{'aerosol': 0.8393999934196472, 'blue': 0.8328999876976013, 'green': 0.805400013923645, 'nir': 0.7477999925613403, 'red': 0.8030999898910522, 'swir': 0.37540000677108765, 'swir2': 0.31200000643730164, 'thermal': 263.8999938964844, 'thermal2': 264.20001220703125}


In [28]:
l8_renamed_scaled_toa = collection.rescale(l8sr_renamed, l8sr, l8toa, renamed=True, drop=False)
print_values(l8_renamed_scaled_toa)

{'aerosol': 0.8393999934196472, 'blue': 0.8328999876976013, 'green': 0.805400013923645, 'nir': 0.7477999925613403, 'pixel_qa': 480, 'radsat_qa': 0, 'red': 0.8030999898910522, 'sr_aerosol': 8, 'swir': 0.37540000677108765, 'swir2': 0.31200000643730164, 'thermal': 263.8999938964844, 'thermal2': 264.20001220703125}


## Re-scale a collection

In [29]:
col = l8sr.collection.limit(5)

In [30]:
col_renamed = col.map(lambda img: l8sr.rename(img))

In [31]:
col_reescaled = col_renamed.map(lambda img: collection.rescale(img, l8sr, l8toa, drop=False, renamed=True))

In [32]:
tools.imagecollection.getValues(col_reescaled, ee.Image(col.first()).geometry().centroid(), 30, side='client')

{'LC08_001004_20140524': {'aerosol': 0.8393999934196472,
  'blue': 0.8328999876976013,
  'green': 0.805400013923645,
  'nir': 0.7477999925613403,
  'pixel_qa': 480,
  'radsat_qa': 0,
  'red': 0.8030999898910522,
  'sr_aerosol': 8,
  'swir': 0.37540000677108765,
  'swir2': 0.31200000643730164,
  'thermal': 263.8999938964844,
  'thermal2': 264.20001220703125},
 'LC08_001004_20140609': {'aerosol': 0.5275999903678894,
  'blue': 0.5042999982833862,
  'green': 0.4708999991416931,
  'nir': 0.4036000072956085,
  'pixel_qa': 480,
  'radsat_qa': 0,
  'red': 0.45190000534057617,
  'sr_aerosol': 8,
  'swir': 0.08799999952316284,
  'swir2': 0.09019999951124191,
  'thermal': 260.79998779296875,
  'thermal2': 260.8999938964844},
 'LC08_001004_20140625': {'aerosol': 0.42309999465942383,
  'blue': 0.4203000068664551,
  'green': 0.41110000014305115,
  'nir': 0.41990000009536743,
  'pixel_qa': 480,
  'radsat_qa': 0,
  'red': 0.4133000075817108,
  'sr_aerosol': 8,
  'swir': 0.39320001006126404,
  'swir2':

# From Id

In [33]:
l5toa = collection.Landsat.fromId('LANDSAT/LT05/C01/T1_TOA')

In [34]:
collection.info(l5toa)

{'spacecraft': 'LANDSAT',
 'id': 'LANDSAT/LT05/C01/T1_TOA',
 'bands': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'BQA'],
 'band_names': ['blue',
  'green',
  'red',
  'nir',
  'swir',
  'thermal',
  'swir2',
  'bqa'],
 'ranges': {'B1': {'min': 0, 'max': 1},
  'B2': {'min': 0, 'max': 1},
  'B3': {'min': 0, 'max': 1},
  'B4': {'min': 0, 'max': 1},
  'B5': {'min': 0, 'max': 1},
  'B6': {'min': 0, 'max': 1},
  'B7': {'min': 0, 'max': 1},
  'BQA': {'min': None, 'max': None}},
 'scales': {'B1': 30,
  'B2': 30,
  'B3': 30,
  'B4': 30,
  'B5': 30,
  'B6': 30,
  'B7': 30,
  'BQA': 30},
 'start_date': '1984-01-01',
 'end_date': '2012-05-05',
 'algorithms': {'brdf': <bound method Landsat.brdf of <geetools.collection.landsat.Landsat object at 0x000001FA38BC9CC8>>,
  'harmonize': <bound method Landsat.harmonize of <geetools.collection.landsat.Landsat object at 0x000001FA38BC9CC8>>,
  'rescale': <bound method Landsat.rescaleAll of <geetools.collection.landsat.Landsat object at 0x000001FA38BC9CC8>>},

In [35]:
l5toa.getBand('B3').name

'red'

## Visualize

In [37]:
Map = ui.Map()

In [38]:
Map.show()

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [39]:
rect = ee.Geometry.Rectangle([-71.57, -40.81, -71.43, -40.72])

In [40]:
Map.addLayer(rect)

In [41]:
p = ee.Geometry.Point([-71.63085937500001, -40.76448635693034])

In [42]:
l8sri = l8sr.collection.filterBounds(p).filterMetadata(l8sr.cloud_cover, 'less_than', 30).first()

In [43]:
vis = l8sr.visualization('NSR')

In [44]:
vis

{'bands': ['B5', 'B6', 'B4'],
 'min': [0, 0, 0],
 'max': [5000.0, 5000.0, 5000.0]}

In [45]:
Map.addLayer(l8sri, vis, 'L8 SR NSR')

In [46]:
Map.centerObject(l8sri)

In [47]:
proxy = l8sr.proxyImage()

In [49]:
ui.eprint(proxy)

## Mask

In [50]:
l8sr.bitOptions()

{'sr_aerosol': ['water', 'climatology', 'low', 'medium', 'high'],
 'pixel_qa': ['clear',
  'water',
  'shadow',
  'snow',
  'cloud',
  'high_confidence_cloud',
  'high_confidence_cirrus'],
 'radsat_qa': ['B1_saturated',
  'B2_saturated',
  'B3_saturated',
  'B4_saturated',
  'B5_saturated',
  'B6_saturated',
  'B7_saturated',
  'B9_saturated',
  'B10_saturated',
  'B11_saturated']}

In [51]:
masked = l8sr.applyMask(l8sri, 'pixel_qa', ['cloud', 'snow', 'shadow'])

In [52]:
Map.addLayer(masked, vis, 'Masked L8 SR (pixel-qa)')